# **Interval computations - Labs**

## **Prerequisites**

In [0]:
!pip install -q pyinterval

In [0]:
from interval import interval, imath
import pandas as pd
import numpy as np
import math
from numpy.linalg import inv

## **Equation**

### Utility

In [0]:
def width(x_i):
    return sum([x.sup - x.inf for x in x_i])

def midpoint(x_i):
    return (x_i[0].sup + x_i[0].inf) / 2

def itos(x_i):
    return f"[{x_i[0].inf:9.7f}, {x_i[0].sup:9.7f}]"

### Bisection method

#### *Bisection with console output*

In [0]:
def bisection(f, start, end, tol=0.01, iteration=1):
    X = interval[start, end]
    F = f(X)
    # No root if no zero
    if 0 not in F:
        print(iteration, '-', itos(X), itos(F), sep='\t')
        return
    # Found root if satisfies the tolerance
    if width(X) < tol:
        print(iteration, '+', itos(X), itos(F), sep='\t')
        return
    
    print(iteration, '*', itos(X), itos(F), sep='\t')
    mid = midpoint(X)
    bisection(f, start, mid, tol, iteration + 1)
    bisection(f, mid, end, tol, iteration + 1)

#### *Bisection with table output*

In [0]:
def bisection_tbl(f, start, end, tol=0.01):
    pd.set_option('display.max_rows', 999)  
    tbl = pd.DataFrame(columns=['Depth', 'Mark', 'Interval', 'Width', 'Function extension'])

    def bisection(f, start, end, tol=0.01, iteration=1):
        X = interval[start, end]
        F = f(X)
        # No root if no zero
        if 0 not in F:
            tbl.loc[len(tbl)] = [iteration, 'no root', itos(X), width(X), itos(F)]
            return
        # Found root if satisfies the tolerance
        if width(X) < tol:
            tbl.loc[len(tbl)] = [iteration, 'RESULT', itos(X), width(X), itos(F)]
            return
        
        tbl.loc[len(tbl)] = [iteration, 'split', itos(X), width(X), itos(F)]

        mid = midpoint(X)
        bisection(f, start, mid, tol, iteration + 1)
        bisection(f, mid, end, tol, iteration + 1)

    bisection(f, start, end, tol)
    return tbl.sort_values(['Depth', 'Interval'])

#### *Example*

In [22]:
func = lambda x: (x**2 + 3*x - 10)*imath.cos(x)
bisection_tbl(func, 0, 3, 1e-2)

,Depth,Mark,Interval,Width,Function extension
0,1,split,"[0.0000000, 3.0000000]",3.000000,"[-10.0000000, 9.8999250]"
1,2,no root,"[0.0000000, 1.5000000]",1.500000,"[-10.0000000, -0.2298959]"
2,2,split,"[1.5000000, 3.0000000]",1.500000,"[-7.9199400, 3.2174756]"
3,3,split,"[1.5000000, 2.2500000]",0.750000,"[-1.1385647, 2.0415643]"
30,3,no root,"[2.2500000, 3.0000000]",0.750000,"[-7.9199400, -1.1385647]"
4,4,split,"[1.5000000, 1.8750000]",0.375000,"[-0.2298959, 0.9734839]"
17,4,split,"[1.8750000, 2.2500000]",0.375000,"[-1.1385647, 0.5398367]"
5,5,split,"[1.5000000, 1.6875000]",0.187500,"[-0.2298959, 0.3784266]"
16,5,no root,"[1.6875000, 1.8750000]",0.187500,"[0.1000647, 0.6259782]"
18,5,split,"[1.8750000, 2.0625000]",0.187500,"[-0.2084004, 0.4057354]"


### Moore's method

#### *Description*

$f(x) = 0$, interval $[a, b]$

$f'(x) \in C^1[a,b]$

$F'(x)$ - inclusion monotonic interval extension of $f'(x)$

$N(X) = m(X) + (- \frac{1}{F'(X)}) f(m(X))$

Necessary conditions:  
$f(x)$ has 1 (non-multiple) root at most on $[a, b]$

#### *Moore's with console output*

In [0]:
def moore(f, df, start, end, tol=1e-6, iteration=1):
    X = interval([start, end])
    X_width = width(X)

    # [Step 1]: Stop if no 0 in F(X)
    if 0 not in f(X):
        print(iteration, '-', itos(X), X_width, sep='\t')
        return
    
    # Return result if tolerance is satisfied
    if X_width < tol:
        print(iteration, '+', itos(X), X_width, sep='\t')
        return
    
    # [Step 2]: Half if 0 in F'(X)
    x_mid = midpoint(X)
    df_X = df(X)
    if 0 in df_X:
        print(iteration, '**', itos(X), X_width, sep='\t')
        moore(f, df, start, x_mid, tol, iteration + 1)
        moore(f, df, x_mid,   end, tol, iteration + 1)
        return
    
    # [Step 3]: Stop if X_next is empty
    f_mid = f(x_mid)
    U = x_mid - f_mid / df_X
    X_next = U & X
    if not X_next:
        print(iteration, '-', itos(U) + '&' + itos(X), sep='\t')
        return

    # [Step 4]: Continue with narrowed interval
    print(iteration, '*', itos(X), X_width, sep='\t')
    moore(f, df, X_next[0].inf, X_next[0].sup, tol, iteration + 1)

#### *Moore's with table output*

In [0]:
def moore_tbl(f, df, start, end, tol=1e-6, iteration=1):
    pd.set_option('display.max_rows', 999)  
    tbl = pd.DataFrame(columns=['Depth', 'Status', 'Interval', 'Width', 'Function extension'])

    def tbl_put(iteration, status, X, X_width, f_X):
        tbl.loc[len(tbl)] = [iteration, status, itos(X), X_width, itos(f_X)]

    def moore(start, end, iteration=1):
        X = interval([start, end])
        X_width = width(X)
        f_X = f(X)

        # [Step 1]: Stop if no 0 in F(X)
        if 0 not in f_X:
            tbl_put(iteration, 'no root', X, X_width, f_X)
            return
        
        # Return result if tolerance is satisfied
        if X_width < tol:
            tbl_put(iteration, 'RESULT', X, X_width, f_X)
            return
        
        # [Step 2]: Half if 0 in F'(X)
        x_mid = midpoint(X)
        df_X = df(X)
        if 0 in df_X:
            tbl_put(iteration, 'split', X, X_width, f_X)
            moore(start, x_mid, iteration + 1)
            moore(x_mid,   end, iteration + 1)
            return
        
        # [Step 3]: Stop if X_next is empty
        f_mid = f(x_mid)
        U = x_mid - f_mid / df_X
        X_next = U & X
        if not X_next:
            tbl_put(iteration, 'empty intersection', X, X_width, f_X)
            return

        # [Step 4]: Continue with narrowed interval
        tbl_put(iteration, 'continue', X, X_width, f_X)
        moore(X_next[0].inf, X_next[0].sup, iteration + 1)

    moore(start, end)
    return tbl.sort_values(['Depth', 'Status', 'Interval'])

#### *Example*

In [24]:
func = lambda x: (x**2 + 3*x - 10)*imath.cos(x)
dfunc = lambda x: (2*x + 3)*imath.cos(x) - (x**2+3*x-10)*imath.sin(x)

moore_tbl(func, dfunc, 0, 3)

,Depth,Status,Interval,Width,Function extension
0,1,split,"[0.0000000, 3.0000000]",3.000000e+00,"[-10.0000000, 9.8999250]"
1,2,no root,"[0.0000000, 1.5000000]",1.500000e+00,"[-10.0000000, -0.2298959]"
2,2,split,"[1.5000000, 3.0000000]",1.500000e+00,"[-7.9199400, 3.2174756]"
16,3,no root,"[2.2500000, 3.0000000]",7.500000e-01,"[-7.9199400, -1.1385647]"
3,3,split,"[1.5000000, 2.2500000]",7.500000e-01,"[-1.1385647, 2.0415643]"
10,4,continue,"[1.8750000, 2.2500000]",3.750000e-01,"[-1.1385647, 0.5398367]"
4,4,split,"[1.5000000, 1.8750000]",3.750000e-01,"[-0.2298959, 0.9734839]"
5,5,continue,"[1.5000000, 1.6875000]",1.875000e-01,"[-0.2298959, 0.3784266]"
11,5,continue,"[1.8891124, 2.0301426]",1.410302e-01,"[-0.0939515, 0.3386921]"
9,5,no root,"[1.6875000, 1.8750000]",1.875000e-01,"[0.1000647, 0.6259782]"


### Hansen's method

#### *Hansen's with console output*

In [0]:
def hansen(f, df, start, end, tol=1e-6, iteration=1):
    X = interval([start, end])
    X_width = width(X)

    # [Step 1]: Stop if no 0 in F(X)
    if 0 not in f(X):
        print(iteration, '-', itos(X), X_width, sep='\t')
        return

    # Return result if tolerance is satisfied
    if X_width < tol:
        print(iteration, "+", itos(X), X_width, sep='\t')
        return
    
    # [Step 2]: Half if 0 in F'(X)
    x_mid = midpoint(X)
    f_mid = f(x_mid)
    df_X = df(X)
    if f_mid == 0.0:
        print(iteration, "+", itos(X), X_width, '<====== { f == 0 }', sep='\t')
        hansen(f, df, start, x_mid, tol, iteration + 1)
        hansen(f, df, x_mid, end, tol, iteration + 1)
        return
    
    # [Step 3]: Stop if X_next is empty
    U = x_mid - f_mid / df_X
    X_next = U & X
    if not X_next:
        print(iteration, '-', itos(U) + '&' + itos(X), sep='\t')
        return

    # [Step 4]: Continue with narrowed intervals
    print(iteration, "*", itos(X), X_width, sep='\t')
    [hansen(f, df, x.inf, x.sup, tol, iteration + 1) for x in X_next]

#### *Hansen's with table output*

In [0]:
def hansen_tbl(f, df, start, end, tol=1e-6, iteration=1):
    tbl = pd.DataFrame(columns=['Depth', 'Status', 'Interval', 'Width', 'Function extension'])

    def tbl_put(iteration, status, X, X_width, f_X):
        tbl.loc[len(tbl)] = [iteration, status, itos(X), X_width, itos(f_X)]

    def hansen(start, end, iteration=1):
        X = interval([start, end])
        X_width = width(X)
        f_X = f(X)

        # [Step 1]: Stop if no 0 in F(X)
        if 0 not in f_X:
            tbl_put(iteration, 'no root', X, X_width, f_X)
            return

        # Return result if tolerance is satisfied
        if X_width < tol:
            tbl_put(iteration, 'RESULT', X, X_width, f_X)
            return
        
        # [Step 2]: Half if 0 in F'(X)
        x_mid = midpoint(X)
        f_mid = f(x_mid)
        df_X = df(X)
        if f_mid == 0.0:
            tbl_put(iteration, 'RESULT and split', X, X_width, f_X)
            hansen(start, x_mid, iteration + 1)
            hansen(x_mid, end, iteration + 1)
            return

        # [Step 3]: Stop if X_next is empty
        U = x_mid - f_mid / df_X
        X_next = U & X
        if not X_next:
            tbl_put(iteration, 'empty intersection', X, X_width, f_X)
            return

        # [Step 4]: Continue with narrowed intervals
        tbl_put(iteration, 'continue', X, X_width, f_X)
        [hansen(x.inf, x.sup, iteration + 1) for x in X_next]
    
    hansen(start, end)
    return tbl.sort_values(['Depth', 'Status', 'Interval'])

#### *Example*

In [27]:
func = lambda x: (x**2 + 3*x - 10)*imath.cos(x)
dfunc = lambda x: (2*x + 3)*imath.cos(x) - (x**2+3*x-10)*imath.sin(x)

hansen_tbl(func, dfunc, 0, 3)

,Depth,Status,Interval,Width,Function extension
0,1,continue,"[0.0000000, 3.0000000]",3.000000e+00,"[-10.0000000, 9.8999250]"
2,2,continue,"[1.5120998, 3.0000000]",1.487900e+00,"[-7.9199400, 3.1454585]"
1,2,no root,"[0.0000000, 1.4864047]",1.486405e+00,"[-10.0000000, -0.2808076]"
3,3,continue,"[1.5120998, 2.1865159]",6.744161e-01,"[-0.7741426, 1.8350112]"
14,3,no root,"[2.5733899, 3.0000000]",4.266101e-01,"[-7.9199400, -3.6601654]"
4,4,continue,"[1.5120998, 1.7706991]",2.585993e-01,"[-0.1863868, 0.6309204]"
9,4,continue,"[1.8999914, 2.1865159]",2.865245e-01,"[-0.7741426, 0.3985407]"
5,5,continue,"[1.5120998, 1.5942189]",8.211916e-02,"[-0.1863868, 0.0744128]"
10,5,continue,"[1.9535385, 2.0181686]",6.463016e-02,"[-0.0551607, 0.1397603]"
6,6,continue,"[1.5677464, 1.5736015]",5.855103e-03,"[-0.0086586, 0.0079636]"


### Krawczyk's method

#### *Krawczyk's with console output*

In [0]:
def kravchyk(f, df, start, end, tol=1e-6, iteration=1):
    X = interval([start, end])
    X_width = width(X)

    # [Step 1]: Stop if no 0 in F(X)
    if 0 not in f(X):
        print(iteration, "-", itos(X), X_width, sep='\t')
        return

    # Return result if tolerance is satisfied
    if X_width < tol:
        print(iteration, "+", itos(X), X_width, sep='\t')
        return

    # Split interval if 0 in F'(X)
    x_mid = midpoint(X)
    df_X = df(X)
    if 0 in df_X:
        print(iteration, "**", itos(X), X_width, sep='\t')
        kravchyk(f, df, start, x_mid, tol, iteration + 1)
        kravchyk(f, df, x_mid,   end, tol, iteration + 1)
        return

    # Stop if X_next is empty
    df_x = df(x_mid)
    K = x_mid - f(x_mid) / df_x + (1 - df(X) / df_x)*(X - x_mid)
    X_next = K & X
    if not X_next:
        print(iteration, "-", itos(K) + '&' + itos(X), sep='\t')
        return

    # Display interval if shrunk or not
    print(iteration, "*", itos(X), X_width, sep='\t')
    kravchyk(f, df, X_next[0].inf, X_next[0].sup, tol, iteration + 1)

#### *Krawczyk's with table output*

In [0]:
def kravchyk_tbl(f, df, start, end, tol=1e-6, iteration=1):
    tbl = pd.DataFrame(columns=['Depth', 'Status', 'Interval', 'Width', 'Function extension'])

    def tbl_put(iteration, status, X, X_width, f_X):
        tbl.loc[len(tbl)] = [iteration, status, itos(X), X_width, itos(f_X)]

    def kravchyk(start, end, iteration=1):
        X = interval([start, end])
        X_width = width(X)
        f_X = f(X)

        # [Step 1]: Stop if no 0 in F(X)
        if 0 not in f_X:
            tbl_put(iteration, 'no root', X, X_width, f_X)
            return

        # Return result if tolerance is satisfied
        if X_width < tol:
            tbl_put(iteration, 'RESULT', X, X_width, f_X)
            return

        # Split interval if 0 in F'(X)
        x_mid = midpoint(X)
        df_X = df(X)
        if 0 in df_X:
            tbl_put(iteration, 'split', X, X_width, f_X)
            kravchyk(start, x_mid, iteration + 1)
            kravchyk(x_mid,   end, iteration + 1)
            return

        # Stop if X_next is empty
        df_x = df(x_mid)
        K = x_mid - f(x_mid) / df_x + (1 - df(X) / df_x)*(X - x_mid)
        X_next = K & X
        if not X_next:
            tbl_put(iteration, 'empty intersection', X, X_width, f_X)
            return

        # Display interval if shrunk or not
        tbl_put(iteration, 'continue', X, X_width, f_X)
        kravchyk(X_next[0].inf, X_next[0].sup, iteration + 1)
    
    kravchyk(start, end)
    return tbl.sort_values(['Depth', 'Status', 'Interval'])

#### *Example*

In [29]:
func = lambda x: (x**2 + 3*x - 10)*imath.cos(x)
dfunc = lambda x: (2*x + 3)*imath.cos(x) - (x**2+3*x-10)*imath.sin(x)

kravchyk_tbl(func, dfunc, 0, 3)

,Depth,Status,Interval,Width,Function extension
0,1,split,"[0.0000000, 3.0000000]",3.000000e+00,"[-10.0000000, 9.8999250]"
1,2,no root,"[0.0000000, 1.5000000]",1.500000e+00,"[-10.0000000, -0.2298959]"
2,2,split,"[1.5000000, 3.0000000]",1.500000e+00,"[-7.9199400, 3.2174756]"
18,3,no root,"[2.2500000, 3.0000000]",7.500000e-01,"[-7.9199400, -1.1385647]"
3,3,split,"[1.5000000, 2.2500000]",7.500000e-01,"[-1.1385647, 2.0415643]"
11,4,continue,"[1.8750000, 2.2500000]",3.750000e-01,"[-1.1385647, 0.5398367]"
4,4,split,"[1.5000000, 1.8750000]",3.750000e-01,"[-0.2298959, 0.9734839]"
5,5,continue,"[1.5000000, 1.6875000]",1.875000e-01,"[-0.2298959, 0.3784266]"
12,5,continue,"[1.8750000, 2.1412407]",2.662407e-01,"[-0.5446686, 0.4640678]"
10,5,no root,"[1.6875000, 1.8750000]",1.875000e-01,"[0.1000647, 0.6259782]"


## **System of equations**

### *Utility*

In [0]:
def print_interval(iteration, status, X, Y):
    print(iteration, status, 'X:'+itos(X), width(X), 'Y:'+itos(Y), width(Y), sep='\t')

# returns determinant, 
def det2x2(m):
    return m[0][0]*m[1][1] - m[0][1]*m[1][0]

# subinterval, t - top, b - bottom, l - left, r - right
def half_interval(x):
    mid = midpoint(x)
    return (interval[x[0].inf, mid], interval[mid, x[0].sup])

### *Moore's method*

$N(\vec{X}) = m(\vec{X}) + (- \frac{1}{F'(\vec{X})}) f(m(\vec{X}))$

#### Moore's with console output

In [0]:
# Example:
# f = lambda x, y: (x + y, x - y)
# df = lambda x, y: ((1,  1),
#                    (1, -1))
# intervals = (interval[-1, 1], interval[-1, 1])
#
def moore_system(f, df, intervals, tol=1e-6, iteration=1):
    X, Y = intervals
    X_width, Y_width = width(X), width(Y)

    # [Step 1]: Stop if no 0 in F(X, Y)
    f_XY = f(X, Y)
    if 0 not in f_XY[0] or 0 not in f_XY[1]:
        print_interval(iteration, '-', X, Y)
        return

    # Return result if tolerance is satisfied
    if max(X_width, Y_width) < tol:
        print_interval(iteration, '+', X, Y)
        return

    # [Step 2]: Half if 0 in F'(X, Y)
    x_mid, y_mid = midpoint(X), midpoint(Y)
    df_XY = df(X, Y)
    df_XY_det = interval(det2x2(df_XY))

    if 0 in df_XY_det:
        print_interval(iteration, '**', X, Y)
        Xl, Xr = half_interval(X)
        Yl, Yr = half_interval(Y)
        if X_width < tol: # half only Y
            moore_system(f, df, (X, Yl), tol, iteration + 1)
            moore_system(f, df, (X, Yr), tol, iteration + 1)
        elif Y_width < tol: # half only X
            moore_system(f, df, (Xl, Y), tol, iteration + 1)
            moore_system(f, df, (Xr, Y), tol, iteration + 1)
        else: # half everything
            moore_system(f, df, (Xl, Yl), tol, iteration + 1)
            moore_system(f, df, (Xl, Yr), tol, iteration + 1)
            moore_system(f, df, (Xr, Yl), tol, iteration + 1)
            moore_system(f, df, (Xr, Yr), tol, iteration + 1)
        return

    f1m, f2m = f(x_mid, y_mid)
    df1x, df1y, df2x, df2y = *df_XY[0], *df_XY[1]
    det = df_XY_det
    U_X = x_mid + ( (-df2y/det)*f1m + ( df1y/det)*f2m )
    U_Y = y_mid + ( ( df2x/det)*f1m + (-df1x/det)*f2m )
    X_next, Y_next = U_X & X, U_Y & Y
    
    # [Step 3]: Stop if X_next or Y_next is empty
    if not X_next or not Y_next:
        print('empty ==> ', end='')
        print_interval(iteration, '-', X, Y)
        return

    # [Step 4]: Continue with narrowed interval
    print_interval(iteration, "*", X, Y)
    moore_system(f, df, (X_next, Y_next), tol, iteration + 1)

#### Moore's with table output

In [0]:
def moore_system_tbl(f, df, intervals, tol=1e-6):
    tbl = pd.DataFrame(columns=['Depth', 'Status', 'X', 'X width', 'Y', 'Y width'])

    def tbl_put(iteration, status, X, Y, X_width, Y_width):
        tbl.loc[len(tbl)] = [iteration, status, itos(X), X_width, itos(Y), Y_width]

    def moore_system(intervals, iteration=1):
        X, Y = intervals
        X_width, Y_width = width(X), width(Y)

        # [Step 1]: Stop if no 0 in F(X, Y)
        f_XY = f(X, Y)
        if 0 not in f_XY[0] or 0 not in f_XY[1]:
            tbl_put(iteration, 'no root', X, Y, X_width, Y_width)
            return

        # Return result if tolerance is satisfied
        if max(X_width, Y_width) < tol:
            tbl_put(iteration, 'RESULT', X, Y, X_width, Y_width)
            return

        # [Step 2]: Half if 0 in F'(X, Y)
        x_mid, y_mid = midpoint(X), midpoint(Y)
        df_XY = df(X, Y)
        df_XY_det = interval(det2x2(df_XY))

        if 0 in df_XY_det:
            Xl, Xr = half_interval(X)
            Yl, Yr = half_interval(Y)
            if X_width < tol: # half only Y
                tbl_put(iteration, 'half Y', X, Y, X_width, Y_width)
                moore_system((X, Yl), iteration + 1)
                moore_system((X, Yr), iteration + 1)
            elif Y_width < tol: # half only X
                tbl_put(iteration, 'half X', X, Y, X_width, Y_width)
                moore_system((Xl, Y), iteration + 1)
                moore_system((Xr, Y), iteration + 1)
            else: # half everything
                tbl_put(iteration, 'half X and Y', X, Y, X_width, Y_width)
                moore_system((Xl, Yl), iteration + 1)
                moore_system((Xl, Yr), iteration + 1)
                moore_system((Xr, Yl), iteration + 1)
                moore_system((Xr, Yr), iteration + 1)
            return

        f1m, f2m = f(x_mid, y_mid)
        df1x, df1y, df2x, df2y = *df_XY[0], *df_XY[1]
        det = df_XY_det
        U_X = x_mid + ( (-df2y/det)*f1m + ( df1y/det)*f2m )
        U_Y = y_mid + ( ( df2x/det)*f1m + (-df1x/det)*f2m )
        X_next, Y_next = U_X & X, U_Y & Y

        # [Step 3]: Stop if X_next or Y_next is empty
        if not X_next or not Y_next:
            tbl_put(iteration, 'empty intersection', X, Y, X_width, Y_width)
            return

        if X == X_next and Y == Y_next:
            if(X_width > Y_width):
                tbl_put(iteration, 'force split X', X, Y, X_width, Y_width)
                Xl, Xr = half_interval(X)
                moore_system((Xl, Y), iteration + 1)
                moore_system((Xr, Y), iteration + 1)
            else:
                tbl_put(iteration, 'force split Y', X, Y, X_width, Y_width)
                Yl, Yr = half_interval(Y)
                moore_system((X, Yl), iteration + 1)
                moore_system((X, Yr), iteration + 1)
            return

        # [Step 4]: Continue with narrowed interval
        tbl_put(iteration, 'continue', X, Y, X_width, Y_width)
        moore_system((X_next, Y_next), iteration + 1)

    moore_system(intervals)
    return tbl.sort_values(['Depth', 'Status', 'X', 'Y'])

#### Example

In [0]:
func = lambda x, y: (x + y, x - y)
dfunc = lambda x, y: ((1,  1), (1, -1))
init_intervals = (interval[-40, 1], interval[-1, 100])
moore_system_tbl(func, dfunc, init_intervals, tol=1e-6)

,Depth,Status,X,X width,Y,Y width
0,1,continue,"[-40.0000000, 1.0000000]",41.0,"[-1.0000000, 100.0000000]",101.0
1,2,RESULT,"[-0.0000000, 0.0000000]",0.0,"[-0.0000000, 0.0000000]",0.0


In [0]:
func = lambda x, y: (x**2 + x - 3*y + 10,
                     x**2 + 3*y - 4*x - 12)
dfunc = lambda x, y: ((2*x+1, -3),
                      (2*x-4,  3))
init_intervals = (interval[-1, 2.5], interval[3, 5.5])
moore_system_tbl(func, dfunc, init_intervals, tol=1e-6)

In [0]:
func = lambda x, y: (x**3 - x**2 + 3 - y,
                     x**2 + 1 - y)
dfunc = lambda x, y: ((3*x**2 - 2*x, -1),
                      (         2*x, -1))
init_intervals = (interval[-0.9, -0.8], interval[1.5, 1.8])
moore_system_tbl(func, dfunc, init_intervals, tol=1e-6)

,Depth,Status,X,X width,Y,Y width
0,1,continue,"[-0.9000000, -0.8000000]",1.000000e-01,"[1.5000000, 1.8000000]",3.000000e-01
1,2,continue,"[-0.8405891, -0.8380579]",2.531167e-03,"[1.6958706, 1.7145996]",1.872896e-02
2,3,continue,"[-0.8392874, -0.8392861]",1.332322e-06,"[1.7043988, 1.7044057]",6.892175e-06
3,4,RESULT,"[-0.8392868, -0.8392868]",4.551914e-15,"[1.7044023, 1.7044023]",3.042011e-14


### *Hansen's method*

#### Hansen's with console output

In [0]:
def hansen_system(f, df, intervals, tol=1e-6, iteration=1):
    X, Y = intervals
    X_width, Y_width = width(X), width(Y)    
    
    # Stop if no 0 in F(X, Y)
    f_XY = f(X, Y)
    if 0 not in f_XY[0] or 0 not in f_XY[1]:
        print_interval(iteration, '-', X, Y)
        return
    
    # Return result if tolerance is satisfied
    if max(X_width, Y_width) < tol:
        print_interval(iteration, '+', X, Y)
        return

    x_mid, y_mid = midpoint(X), midpoint(Y)
    df_XY = df(X, Y)
    df_XY_det = interval(det2x2(df_XY))  
    f1m, f2m = f(x_mid, y_mid)

    # Check midpoints
    if f1m == 0.0 and f2m == 0.0:
        print('Result with split ==>', end=' ')
        print_interval(iteration, '+', X, Y)
        Xl, Xr = half_interval(X)
        Yl, Yr = half_interval(Y)
        hansen_system(f, df, (Xl, Yl), tol, iteration + 1)
        hansen_system(f, df, (Xl, Yr), tol, iteration + 1)
        hansen_system(f, df, (Xr, Yl), tol, iteration + 1)
        hansen_system(f, df, (Xr, Yr), tol, iteration + 1)
        return
 
    df1x, df1y, df2x, df2y = *df_XY[0], *df_XY[1]
    det = df_XY_det
    U_X = x_mid + ( (-df2y/det)*f1m + ( df1y/det)*f2m )
    U_Y = y_mid + ( ( df2x/det)*f1m + (-df1x/det)*f2m )
    X_next, Y_next = U_X & X, U_Y & Y

    # Stop if X_next is empty
    if not X_next or not Y_next:
        print('empty ==>', end=' ')
        print_interval(iteration, '-', X, Y)
        return

    # Force split if X and Y haven't changed
    if X == X_next and Y == Y_next:
        print_interval(iteration, '**', X, Y)
        if(X_width > Y_width):
            print('force split X')
            Xl, Xr = half_interval(X)
            hansen_system(f, df, (Xl, Y), tol, iteration + 1)
            hansen_system(f, df, (Xr, Y), tol, iteration + 1)
        else:
            print('force split Y')
            Yl, Yr = half_interval(Y)
            hansen_system(f, df, (X, Yl), tol, iteration + 1)
            hansen_system(f, df, (X, Yr), tol, iteration + 1)
        return
    
    # Continue with narrowed intervals
    print_interval(iteration, "*", X_next, Y_next)
    for Xi in X_next:
        for Yi in Y_next:
            hansen_system(f, df, (interval(Xi), interval(Yi)), tol, iteration + 1)

#### Hansen's with table output

In [0]:
def hansen_system_tbl(f, df, intervals, tol=1e-6):
    tbl = pd.DataFrame(columns=['Depth', 'Status', 'X', 'X width', 'Y', 'Y width'])

    def tbl_put(iteration, status, X, Y, X_width, Y_width):
        tbl.loc[len(tbl)] = [iteration, status, itos(X), X_width, itos(Y), Y_width]

    def hansen_system(intervals, iteration=1):
        X, Y = intervals
        X_width, Y_width = width(X), width(Y)    

        # Stop if no 0 in F(X, Y)
        f_XY = f(X, Y)
        if 0 not in f_XY[0] or 0 not in f_XY[1]:
            tbl_put(iteration, 'no root', X, Y, X_width, Y_width)
            return

        # Return result if tolerance is satisfied
        if max(X_width, Y_width) < tol:
            tbl_put(iteration, 'RESULT', X, Y, X_width, Y_width)
            return

        x_mid, y_mid = midpoint(X), midpoint(Y)
        df_XY = df(X, Y)
        df_XY_det = interval(det2x2(df_XY))  
        f1m, f2m = f(x_mid, y_mid)

        # Check midpoints
        if f1m == 0.0 and f2m == 0.0:
            tbl_put(iteration, 'RESULT with split', X, Y, X_width, Y_width)
            Xl, Xr = half_interval(X)
            Yl, Yr = half_interval(Y)
            hansen_system((Xl, Yl), iteration + 1)
            hansen_system((Xl, Yr), iteration + 1)
            hansen_system((Xr, Yl), iteration + 1)
            hansen_system((Xr, Yr), iteration + 1)
            return

        df1x, df1y, df2x, df2y = *df_XY[0], *df_XY[1]
        det = df_XY_det
        U_X = x_mid + ( (-df2y/det)*f1m + ( df1y/det)*f2m )
        U_Y = y_mid + ( ( df2x/det)*f1m + (-df1x/det)*f2m )
        X_next, Y_next = U_X & X, U_Y & Y

        # Stop if X_next is empty
        if not X_next or not Y_next:
            tbl_put(iteration, 'empty intersection', X, Y, X_width, Y_width)
            return

        # Force split if X and Y haven't changed
        if X == X_next and Y == Y_next:
            if(X_width > Y_width):
                tbl_put(iteration, 'force split X', X, Y, X_width, Y_width)
                Xl, Xr = half_interval(X)
                hansen_system((Xl, Y), iteration + 1)
                hansen_system((Xr, Y), iteration + 1)
            else:
                tbl_put(iteration, 'force split Y', X, Y, X_width, Y_width)
                Yl, Yr = half_interval(Y)
                hansen_system((X, Yl), iteration + 1)
                hansen_system((X, Yr), iteration + 1)
            return

        # Continue with narrowed intervals
        tbl_put(iteration, 'continue', X, Y, X_width, Y_width)
        for Xi in X_next:
            for Yi in Y_next:
                hansen_system((interval(Xi), interval(Yi)), iteration + 1)

    hansen_system(intervals)
    return tbl.sort_values(['Depth', 'Status', 'X', 'Y'])

#### Example

In [0]:
func = lambda x, y: (x + y, x - y)
dfunc = lambda x, y: ((1,  1), (1, -1))
init_intervals = (interval[-40, 1], interval[-1, 100])
hansen_system_tbl(func, dfunc, init_intervals, tol=1e-6)

,Depth,Status,X,X width,Y,Y width
0,1,continue,"[-40.0000000, 1.0000000]",41.0,"[-1.0000000, 100.0000000]",101.0
1,2,RESULT,"[-0.0000000, -0.0000000]",0.0,"[-0.0000000, -0.0000000]",0.0


In [0]:
func = lambda x, y: (x**2 + x - 3*y + 10,
                     x**2 + 3*y - 4*x - 12)
dfunc = lambda x, y: ((2*x+1, -3),
                      (2*x-4,  3))
init_intervals = (interval[-1, 2.5], interval[3, 5.5])
hansen_system_tbl(func, dfunc, init_intervals, tol=1e-6)

,Depth,Status,X,X width,Y,Y width
0,1,force split X,"[-1.0000000, 2.5000000]",3.500000e+00,"[3.0000000, 5.5000000]",2.500000e+00
1,2,force split Y,"[-1.0000000, 0.7500000]",1.750000e+00,"[3.0000000, 5.5000000]",2.500000e+00
15,2,force split Y,"[0.7500000, 2.5000000]",1.750000e+00,"[3.0000000, 5.5000000]",2.500000e+00
2,3,force split X,"[-1.0000000, 0.7500000]",1.750000e+00,"[3.0000000, 4.2500000]",1.250000e+00
16,3,force split X,"[0.7500000, 2.5000000]",1.750000e+00,"[3.0000000, 4.2500000]",1.250000e+00
23,3,force split X,"[0.7500000, 2.5000000]",1.750000e+00,"[4.2500000, 5.5000000]",1.250000e+00
14,3,no root,"[-1.0000000, 0.7500000]",1.750000e+00,"[4.2500000, 5.5000000]",1.250000e+00
3,4,continue,"[-1.0000000, -0.1250000]",8.750000e-01,"[3.0000000, 4.2500000]",1.250000e+00
29,4,continue,"[1.6250000, 2.5000000]",8.750000e-01,"[4.2500000, 5.5000000]",1.250000e+00
7,4,force split Y,"[-0.1250000, 0.7500000]",8.750000e-01,"[3.0000000, 4.2500000]",1.250000e+00


### *Krawczyk's method*

#### Krawczyk's with console output

In [0]:
def kravchyk_system(f, df, intervals, tol=1e-6, iteration=1):
    X, Y = intervals
    X_width, Y_width = width(X), width(Y)

    # Stop if no 0 in F(X, Y)
    f_XY = f(X, Y)
    if 0 not in f_XY[0] or 0 not in f_XY[1]:
        print_interval(iteration, '-', X, Y)
        return

    # Return result if tolerance is satisfied
    if max(X_width, Y_width) < tol:
        print_interval(iteration, '+', X, Y)
        return

    # Half if 0 in F'(X, Y)
    x_mid, y_mid = midpoint(X), midpoint(Y)
    df_XY = df(X, Y)
    df_XY_det = interval(det2x2(df_XY))

    if 0 in df_XY_det:
        Xl, Xr = half_interval(X)
        Yl, Yr = half_interval(Y)
        if X_width < tol: # half only Y
            print_interval(iteration, '**', X, Y)
            kravchyk_system(f, df, (X, Yl), tol, iteration + 1)
            kravchyk_system(f, df, (X, Yr), tol, iteration + 1)
        elif Y_width < tol: # half only X
            print_interval(iteration, '**', X, Y)
            kravchyk_system(f, df, (Xl, Y), tol, iteration + 1)
            kravchyk_system(f, df, (Xr, Y), tol, iteration + 1)
        else: # half everything
            print_interval(iteration, '****', X, Y)
            kravchyk_system(f, df, (Xl, Yl), tol, iteration + 1)
            kravchyk_system(f, df, (Xl, Yr), tol, iteration + 1)
            kravchyk_system(f, df, (Xr, Yl), tol, iteration + 1)
            kravchyk_system(f, df, (Xr, Yr), tol, iteration + 1)
        return

    f1m, f2m = f(x_mid, y_mid)
    df_mid = df(x_mid, y_mid)
    df1x,   df1y,  df2x,  df2y =  *df_XY[0], *df_XY[1]
    df1xm, df1ym, df2xm, df2ym = *df_mid[0], *df_mid[1]
    det = det2x2(df_mid)
    retard_X = (1 + df2x*df1ym/det - df1x*df2ym/det)*(X-x_mid) \
             + (    df2y*df1ym/det - df1y*df2ym/det)*(Y-y_mid)
    retard_Y = (  - df2x*df1xm/det + df1x*df2xm/det)*(X-x_mid) \
             + (1 - df2y*df1xm/det + df1y*df2xm/det)*(Y-y_mid)
    K_X = x_mid + ( (-df2ym/det)*f1m + ( df1ym/det)*f2m ) + retard_X
    K_Y = y_mid + ( ( df2xm/det)*f1m + (-df1xm/det)*f2m ) + retard_Y
    X_next, Y_next = K_X & X, K_Y & Y
    
    # Stop if X_next or Y_next is empty
    if not X_next or not Y_next:
        print('empty ==> ', end='')
        print_interval(iteration, '-', X, Y)
        return

    # Continue with narrowed interval
    print_interval(iteration, "*", X, Y)
    kravchyk_system($1 iteration + 1)

#### Krawczyk's with table output

In [0]:
def kravchyk_system_tbl(f, df, intervals, tol=1e-6, iteration=1):
    tbl = pd.DataFrame(columns=['Depth', 'Status', 'X', 'X width', 'Y', 'Y width'])

    def tbl_put(iteration, status, X, Y, X_width, Y_width):
        tbl.loc[len(tbl)] = [iteration, status, itos(X), X_width, itos(Y), Y_width]

    def kravchyk_system(intervals, iteration=1):
        X, Y = intervals
        X_width, Y_width = width(X), width(Y)

        # Stop if no 0 in F(X, Y)
        f_XY = f(X, Y)
        if 0 not in f_XY[0] or 0 not in f_XY[1]:
            tbl_put(iteration, 'no root', X, Y, X_width, Y_width)
            return

        # Return result if tolerance is satisfied
        if max(X_width, Y_width) < tol:
            tbl_put(iteration, 'RESULT', X, Y, X_width, Y_width)
            return

        # Half if 0 in F'(X, Y)
        x_mid, y_mid = midpoint(X), midpoint(Y)
        df_XY = df(X, Y)
        df_XY_det = interval(det2x2(df_XY))

        if 0 in df_XY_det:
            Xl, Xr = half_interval(X)
            Yl, Yr = half_interval(Y)
            if X_width < tol: # half only Y
                tbl_put(iteration, 'half Y', X, Y, X_width, Y_width)
                kravchyk_system((X, Yl), iteration + 1)
                kravchyk_system((X, Yr), iteration + 1)
            elif Y_width < tol: # half only X
                tbl_put(iteration, 'half X', X, Y, X_width, Y_width)
                kravchyk_system((Xl, Y), iteration + 1)
                kravchyk_system((Xr, Y), iteration + 1)
            else: # half everything
                tbl_put(iteration, 'half X and Y', X, Y, X_width, Y_width)
                kravchyk_system((Xl, Yl), iteration + 1)
                kravchyk_system((Xl, Yr), iteration + 1)
                kravchyk_system((Xr, Yl), iteration + 1)
                kravchyk_system((Xr, Yr), iteration + 1)
            return

        f1m, f2m = f(x_mid, y_mid)
        df_mid = df(x_mid, y_mid)
        df1x,   df1y,  df2x,  df2y =  *df_XY[0], *df_XY[1]
        df1xm, df1ym, df2xm, df2ym = *df_mid[0], *df_mid[1]
        det = det2x2(df_mid)
        retard_X = (1 + df2x*df1ym/det - df1x*df2ym/det)*(X-x_mid) \
                + (    df2y*df1ym/det - df1y*df2ym/det)*(Y-y_mid)
        retard_Y = (  - df2x*df1xm/det + df1x*df2xm/det)*(X-x_mid) \
                + (1 - df2y*df1xm/det + df1y*df2xm/det)*(Y-y_mid)
        K_X = x_mid + ( (-df2ym/det)*f1m + ( df1ym/det)*f2m ) + retard_X
        K_Y = y_mid + ( ( df2xm/det)*f1m + (-df1xm/det)*f2m ) + retard_Y
        X_next, Y_next = K_X & X, K_Y & Y
        
        # Stop if X_next or Y_next is empty
        if not X_next or not Y_next:
            tbl_put(iteration, 'empty intersection', X, Y, X_width, Y_width)
            return

        # Continue with narrowed interval
        tbl_put(iteration, 'continue', X, Y, X_width, Y_width)
        kravchyk_system((X_next, Y_next), iteration + 1)
    
    kravchyk_system(intervals)
    return tbl.sort_values(['Depth', 'Status', 'X', 'Y'])

#### Example

In [0]:
func = lambda x, y: (x + y, x - y)
dfunc = lambda x, y: ((1,  1), (1, -1))
init_intervals = (interval[-40, 1], interval[-1, 100])
kravchyk_system_tbl(func, dfunc, init_intervals, tol=1e-6)

,Depth,Status,X,X width,Y,Y width
0,1,continue,"[-40.0000000, 1.0000000]",41.0,"[-1.0000000, 100.0000000]",101.0
1,2,RESULT,"[-0.0000000, 0.0000000]",0.0,"[-0.0000000, 0.0000000]",0.0


In [0]:
func = lambda x, y: (x**2 + x - 3*y + 10,
                     x**2 + 3*y - 4*x - 12)
dfunc = lambda x, y: ((2*x+1, -3),
                      (2*x-4,  3))
init_intervals = (interval[-1, 2.5], interval[3, 5.5])
kravchyk_system_tbl(func, dfunc, init_intervals, tol=1e-6)

,Depth,Status,X,X width,Y,Y width
0,1,half X and Y,"[-1.0000000, 2.5000000]",3.500000e+00,"[3.0000000, 5.5000000]",2.500000e+00
1,2,half X and Y,"[-1.0000000, 0.7500000]",1.750000e+00,"[3.0000000, 4.2500000]",1.250000e+00
15,2,half X and Y,"[0.7500000, 2.5000000]",1.750000e+00,"[3.0000000, 4.2500000]",1.250000e+00
24,2,half X and Y,"[0.7500000, 2.5000000]",1.750000e+00,"[4.2500000, 5.5000000]",1.250000e+00
14,2,no root,"[-1.0000000, 0.7500000]",1.750000e+00,"[4.2500000, 5.5000000]",1.250000e+00
2,3,continue,"[-1.0000000, -0.1250000]",8.750000e-01,"[3.0000000, 3.6250000]",6.250000e-01
32,3,continue,"[1.6250000, 2.5000000]",8.750000e-01,"[4.8750000, 5.5000000]",6.250000e-01
7,3,empty intersection,"[-1.0000000, -0.1250000]",8.750000e-01,"[3.6250000, 4.2500000]",6.250000e-01
31,3,empty intersection,"[1.6250000, 2.5000000]",8.750000e-01,"[4.2500000, 4.8750000]",6.250000e-01
9,3,half X and Y,"[-0.1250000, 0.7500000]",8.750000e-01,"[3.6250000, 4.2500000]",6.250000e-01


In [0]:
func = lambda x, y: (x**3 - x**2 + 3 - y,
                     x**2 + 1 - y)
dfunc = lambda x, y: ((3*x**2 - 2*x, -1),
                      (         2*x, -1))
init_intervals = (interval[-0.9, -0.8], interval[1.5, 1.8])
kravchyk_system_tbl(func, dfunc, init_intervals, tol=1e-6)

,Depth,Status,X,X width,Y,Y width
0,1,continue,"[-0.9000000, -0.8000000]",1.000000e-01,"[1.5000000, 1.8000000]",3.000000e-01
1,2,continue,"[-0.8435339, -0.8352268]",8.307140e-03,"[1.6954389, 1.7134542]",1.801527e-02
2,3,continue,"[-0.8393153, -0.8392582]",5.706617e-05,"[1.7043410, 1.7044635]",1.224604e-04
3,4,RESULT,"[-0.8392868, -0.8392868]",2.689546e-09,"[1.7044023, 1.7044023]",5.772610e-09


# **THE END**